In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from app.core.config import settings
from app.core.services.intent_parser import AgentIntentParser


# Initialize Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    google_api_key=settings.google_api_key
)

# Initialize parser
parser = AgentIntentParser(llm)

# Test with a relevant query
query = "What is abhiyan working on currently?"

intent = await parser.parse_intent(query, [])
print(intent)



> Entering new AgentExecutor chain...
```json
{
  "is_relevant": true,
  "agent_intent": {
    "intent": "Find current tasks and activities for Abhiyan Timilsina across JIRA and GitHub.",
    "operations": [
      {
        "tool": "search_jira_users",
        "action": "Search for JIRA users matching 'Abhiyan Timilsina'",
        "filters": {
          "query": "abhiyan timilsina",
          "max_results": 10
        },
        "output_keys": ["jira_user_key", "jira_username", "jira_display_name"]
      },
      {
        "tool": "search_jira_issues",
        "action": "Find JIRA issues assigned to Abhiyan",
        "filters": {
          "assignee": "jira_username",
          "status": "Open",
          "max_results": 10
        },
        "output_keys": ["issue_key", "summary", "status", "project_key"]
      },
      {
        "tool": "get_github_recent_activities",
        "action": "Get Abhiyan's recent GitHub activities",
        "filters": {
          "usernames": ["abhiyan-ti

In [3]:
intent.operations

[OperationStep(tool='search_jira_users', action="Search for JIRA users matching 'Abhiyan Timilsina'", filters={'query': 'abhiyan timilsina', 'max_results': 10}, sort_by=None, order=None, output_keys=['jira_user_key', 'jira_username', 'jira_display_name'], aggregation=None),
 OperationStep(tool='search_jira_issues', action='Find JIRA issues assigned to Abhiyan', filters={'assignee': 'jira_username', 'status': 'Open', 'max_results': 10}, sort_by=None, order=None, output_keys=['issue_key', 'summary', 'status', 'project_key'], aggregation=None),
 OperationStep(tool='get_github_recent_activities', action="Get Abhiyan's recent GitHub activities", filters={'usernames': ['abhiyan-timilsina', 'abhiyantimilsina', 'atimilsina', 'abhiyan'], 'days': 7, 'include_commits': True, 'include_prs': True}, sort_by=None, order=None, output_keys=['username', 'activity_type', 'repository', 'date'], aggregation=None)]

In [7]:
from app.clients.github_client import GitHubClient
from app.clients.jira_client import JiraClient

client = GitHubClient()
context = client.context or {}

# Safely extract repository names if available
repos = context.get("repositories", [])
names = ",".join(repo.get("name", "") for repo in repos if repo.get("name"))

# Safely extract contributors if available
contributors = context.get("contributors", [])
members = ",".join(contributor.get("name", "") for contributor in contributors if contributor.get("name"))

context_str = f"""
Repository Names: {names}
Members: {members}
"""
print(context_str)


client = GitHubClient()
context = client.context or {}

# Safely extract repository names if available
repos = context.get("repositories", [])
names = ",".join(repo.get("name", "") for repo in repos if repo.get("name"))

# Safely extract contributors if available
contributors = context.get("contributors", [])
members = ",".join(contributor.get("name", "") for contributor in contributors if contributor.get("name"))

context_str = f"""
Repository Names: {names}
Members: {members}
"""
print(context_str)

client = GitHubClient()
context = client.context or {}



AttributeError: 'list' object has no attribute 'get'

In [10]:
from app.clients.github_client import GitHubClient
from app.clients.jira_client import JiraClient

client = GitHubClient()
context = client.context or []

# Get all repository names
names = ",".join(repo['full_name'] for repo in context if 'name' in repo)

# Get all contributors across repos
members = ",".join(
    contributor['name']
    for repo in context
    for contributor in repo.get('contributors', [])
    if 'name' in contributor
)

context_str = f"""
Repository Names: {names}
Members: {members}
"""
print(context_str)



Repository Names: abhiyan52/team-activity-monitors
Members: Abhiyan Timilsina



In [11]:
jira_client = JiraClient()
context = jira_client.context
project_names = [kcontext['project_information']]


{'project_information': [{'key': 'LEARNJIRA',
   'name': '(Learn) Jira Premium benefits in 5 min 👋',
   'id': '10001',
   'lead': None,
   'users': [{'accountId': '5cac3c474353071e5657aa89',
     'displayName': 'abhiyan timilsina',
     'emailAddress': 'abhiyantimilsina@gmail.com',
     'active': True}]},
  {'key': 'SCRUM',
   'name': 'Team Activity Monitor',
   'id': '10000',
   'lead': None,
   'users': [{'accountId': '5cac3c474353071e5657aa89',
     'displayName': 'abhiyan timilsina',
     'emailAddress': 'abhiyantimilsina@gmail.com',
     'active': True}]}]}

In [2]:
from app.core.config import settings
from langchain_google_genai import ChatGoogleGenerativeAI
from app.core.services.intent_parser import AgentIntentParser

llm =  ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=settings.google_api_key,
    temperature=0
)
intent_parser = AgentIntentParser(llm)

# chat_history = []
# query = "What is abhiyan working on this month?"

# result = await intent_parser.parse_intent(query, chat_history)
# result

In [3]:
chat_history = []
query = "What is abhiyan working on this month?"

result = await intent_parser.parse_intent(query, chat_history)
result

'```json\n{\n  "is_relevant": true,\n  "intent": "Find recent activities for Abhiyan",\n  "operations": [\n    {\n      "tool": "get_recent_activity",\n      "action": "Get activity for Abhiyan across JIRA and GitHub",\n      "filters": {\n        "team_members": [\n          "abhiyan timilsina",\n          "abhiyan52"\n        ],\n        "days": 30\n      },\n      "output_keys": [\n        "jira_activity",\n        "github_activity"\n      ]\n    }\n  ],\n  "members": [\n    "abhiyan timilsina",\n    "abhiyan52"\n  ],\n  "projects": [],\n  "repositories": [],\n  "time_range": {\n    "start": null,\n    "end": null,\n    "label": "this month"\n  },\n  "context": {\n    "user_matching": "abhiyan -> abhiyan timilsina, abhiyan52",\n    "notes": "Cross-platform activity search for the current month"\n  },\n  "error": null\n}\n```'

In [4]:
chat_history = []
query = "What all commits have been done today by Abhiyan?"

result = await intent_parser.parse_intent(query, chat_history)
result

'```json\n{\n  "is_relevant": true,\n  "intent": "Get GitHub commits by Abhiyan for today",\n  "operations": [\n    {\n      "tool": "get_github_commits",\n      "action": "Retrieve commits by abhiyan52",\n      "filters": {\n        "author": "abhiyan52",\n        "since_days": 1\n      },\n      "output_keys": [\n        "commits"\n      ]\n    }\n  ],\n  "members": [\n    "abhiyan52"\n  ],\n  "projects": [],\n  "repositories": [],\n  "time_range": {\n    "start": null,\n    "end": null,\n    "label": "today"\n  },\n  "context": {\n    "user_matching": "Abhiyan -> abhiyan52",\n    "notes": "GitHub-specific query for commits"\n  },\n  "error": null\n}\n```'